In [1]:
import numpy as np 
import pandas as pd

In [2]:
df = pd.read_csv("diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [6]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [7]:
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
scaler = StandardScaler()

In [10]:
X_scaled = scaler.fit_transform(X)

In [11]:
X_scaled

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [12]:
X_scaled.shape

(768, 8)

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_scaled ,y , test_size=0.2,random_state=42)

In [81]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [15]:
model = Sequential()
model.add(Dense(32,activation = 'relu',input_dim = 8))
model.add(Dense(1,activation = 'sigmoid'))

In [16]:
model.compile(loss = 'binary_crossentropy',optimizer = 'Adam',metrics=['accuracy'])
model.fit(X_train,y_train,epochs = 100,batch_size = 32,validation_data = (X_test,y_test))


Epoch 1/100


20/20 [==============================] - 2s 23ms/step - loss: 0.6844 - accuracy: 0.5961 - val_loss: 0.6462 - val_accuracy: 0.6364
Epoch 2/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6339 - accuracy: 0.6482 - val_loss: 0.6074 - val_accuracy: 0.6364
Epoch 3/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5965 - accuracy: 0.6694 - val_loss: 0.5793 - val_accuracy: 0.6623
Epoch 4/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5677 - accuracy: 0.7068 - val_loss: 0.5593 - val_accuracy: 0.6818
Epoch 5/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5461 - accuracy: 0.7150 - val_loss: 0.5436 - val_accuracy: 0.7208
Epoch 6/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5281 - accuracy: 0.7296 - val_loss: 0.5323 - val_accuracy: 0.7273
Epoch 7/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5153 - accuracy: 0.7410 - val_loss: 0.5225 - val_accuracy: 0.7338
Epoch 8/1

In [17]:
import keras_tuner as kt

In [18]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32,activation = 'relu',input_dim = 8))
    model.add(Dense(1,activation = 'sigmoid'))
    
    
    optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])
    model.compile(optimizer = optimizer ,loss = 'binary_crossentropy',metrics = ['accuracy'])
                  
    return model

In [21]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)


Reloading Tuner from .\untitled_project\tuner0.json


In [22]:
tuner.search(X_train,y_train,epochs =5,validation_data = (X_test,y_test))


In [23]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'sgd'}

In [25]:
model1 = tuner.get_best_models(num_models=1)[0]

In [28]:
model1.fit(X_train,y_train,batch_size = 32,epochs =100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 0s 20ms/step - loss: 0.4402 - accuracy: 0.7948 - val_loss: 0.5003 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4401 - accuracy: 0.7948 - val_loss: 0.5004 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4397 - accuracy: 0.7948 - val_loss: 0.5002 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4396 - accuracy: 0.7932 - val_loss: 0.5005 - val_accuracy: 0.7857
Epoch 11/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4394 - accuracy: 0.7932 - val_loss: 0.5009 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4390 - accuracy: 0.7948 - val_loss: 0.5011 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4389 - accuracy: 0.7932 - val_loss: 0.5013 - val_accuracy: 0.7857
Epoch 14

In [42]:
def build_model1(hp):
    model = Sequential()
    units = hp.Int('units',8,128,step = 8)
    
    model.add(Dense(units=units,activation = 'relu',input_dim = 8))
    model.add(Dense(1,activation = 'sigmoid'))
    
    model.compile(optimizer ='sgd',loss ='binary_crossentropy',metrics = ['accuracy'])
    return model

In [43]:
tuner = kt.RandomSearch(build_model1,
                        objective='val_accuracy',
                        max_trials=5,
                       directory = 'my_dir',
                       project_name = 'tuner')

In [44]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7272727489471436

Best val_accuracy So Far: 0.7532467246055603
Total elapsed time: 00h 00m 13s


In [46]:
tuner.get_best_hyperparameters()[0].values

{'units': 112}

In [50]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(72,activation ='relu',input_dim = 8))
            
    for i in range(hp.Int('num_layers',min_value =1,max_value = 10)):
        model.add(Dense(72,activation = 'relu'))
              
    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(optimizer = 'rmsprop',loss = 'binary_crossentropy',metrics=['accuracy'])
    
    return model
    
              

In [51]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory = 'mydir',
                        project_name = 'layer_optimizer')

In [52]:
tuner.search(X_train,y_train,epochs = 5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 20s


In [53]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8}

In [85]:
def build_model(hp):
    model = Sequential()

    counter = 0
    
    for i in range(hp.Int('final',min_value = 1,max_value=10)):
        if counter == 0:
            
            model.add(Dense(hp.Int('units'+str(i),min_value = 8,max_value = 128,step = 8),
                      activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid']),
                      input_dim = 8))
            model.add(Dropout(hp.Choice('dropout'+str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
            model.add(Dense(hp.Int('units'+str(i),min_value = 8,max_value = 128,step = 16),
                      activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid'])))
            model.add(Dropout(hp.Choice('dropout'+str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter +=1   
    model.add(Dense(1,activation = 'sigmoid'))
    
    model.compile(loss = 'binary_crossentropy',optimizer = hp.Choice('optimizer',values= ['adam','sgd','rmsprop','adadelta']),metrics=['accuracy'])
    
    return model
            

In [86]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory = 'my_diir',
                        project_name = 'Final2')

In [87]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7207792401313782

Best val_accuracy So Far: 0.7207792401313782
Total elapsed time: 00h 00m 21s


In [88]:
tuner.get_best_hyperparameters()[0].values




{'final': 3,
 'units0': 48,
 'activation0': 'relu',
 'dropout0': 0.4,
 'optimizer': 'adam',
 'units1': 72,
 'activation1': 'tanh',
 'dropout1': 0.3,
 'units2': 40,
 'activation2': 'sigmoid',
 'dropout2': 0.5,
 'units3': 56,
 'activation3': 'tanh',
 'dropout3': 0.2,
 'units4': 56,
 'activation4': 'sigmoid',
 'dropout4': 0.1,
 'units5': 40,
 'activation5': 'tanh',
 'dropout5': 0.3,
 'units6': 24,
 'activation6': 'tanh',
 'dropout6': 0.7}

In [89]:
model = tuner.get_best_models(num_models=1)[0]

In [90]:
model.fit(X_train,y_train,epochs = 200,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/200
20/20 [==============================] - 2s 21ms/step - loss: 0.5569 - accuracy: 0.7020 - val_loss: 0.5278 - val_accuracy: 0.7143
Epoch 7/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5437 - accuracy: 0.7036 - val_loss: 0.5214 - val_accuracy: 0.7078
Epoch 8/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5369 - accuracy: 0.7492 - val_loss: 0.5161 - val_accuracy: 0.7078
Epoch 9/200
20/20 [==============================] - 0s 7ms/step - loss: 0.5167 - accuracy: 0.7329 - val_loss: 0.5131 - val_accuracy: 0.7338
Epoch 10/200
20/20 [==============================] - 0s 7ms/step - loss: 0.5073 - accuracy: 0.7378 - val_loss: 0.5048 - val_accuracy: 0.7532
Epoch 11/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5139 - accuracy: 0.7541 - val_loss: 0.5035 - val_accuracy: 0.7468
Epoch 12/200
20/20 [==============================] - 0s 6ms/step - loss: 0.4974 - accuracy: 0.7443 - val_loss: 0.5022 - val_accuracy: 0.7532
Epoch 13/